Для произвольного набора данных, предназначенного для классификации текстов, решите задачу классификации текста двумя способами:

    Способ 1. На основе CountVectorizer или TfidfVectorizer.
    Способ 2. На основе моделей word2vec или Glove или fastText.
    Сравните качество полученных моделей.
Для поиска наборов данных в поисковой системе можно использовать ключевые слова "datasets for text classification".

In [1]:
!pip install tensorflow==1.14.0 tensorflow-gpu==1.14.0

You should consider upgrading via the 'c:\users\zenbook\documents\lab1\.venv\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
# 20 Newsgroups - набор состоящий из 20 тясяч постов по 20 различным темам.
from sklearn.datasets import fetch_20newsgroups

import numpy as np
import pandas as pd
from typing import Dict, Tuple
from scipy import stats
from IPython.display import Image
from sklearn.datasets import load_iris, load_boston
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score 
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC, NuSVC, LinearSVC, OneClassSVM, SVR, NuSVR, LinearSVR
import seaborn as sns
import matplotlib.pyplot as plt

import gensim
from gensim.models import word2vec
import re
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from nltk import WordPunctTokenizer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

%matplotlib inline 
sns.set(style="ticks")

ModuleNotFoundError: No module named 'sklearn'

In [2]:
def accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray) -> Dict[int, float]:
    """
    Вычисление метрики accuracy для каждого класса
    y_true - истинные значения классов
    y_pred - предсказанные значения классов
    Возвращает словарь: ключ - метка класса, 
    значение - Accuracy для данного класса
    """
    # Для удобства фильтрации сформируем Pandas DataFrame 
    d = {'t': y_true, 'p': y_pred}
    df = pd.DataFrame(data=d)
    # Метки классов
    classes = np.unique(y_true)
    # Результирующий словарь
    res = dict()
    # Перебор меток классов
    for c in classes:
        # отфильтруем данные, которые соответствуют 
        # текущей метке класса в истинных значениях
        temp_data_flt = df[df['t']==c]
        # расчет accuracy для заданной метки класса
        temp_acc = accuracy_score(
            temp_data_flt['t'].values, 
            temp_data_flt['p'].values)
        # сохранение результата в словарь
        res[c] = temp_acc
    return res

def print_accuracy_score_for_classes(
    y_true: np.ndarray, 
    y_pred: np.ndarray):
    """
    Вывод метрики accuracy для каждого класса
    """
    accs = accuracy_score_for_classes(y_true, y_pred)
    if len(accs)>0:
        print('Метка \t Accuracy')
    for i in accs:
        print('{} \t {}'.format(i, accs[i]))

In [3]:
categories = ["comp.sys.ibm.pc.hardware", "rec.motorcycles", "sci.electronics", "alt.atheism"]
# Извлекаем из набора подмножество - для обучения
newsgroups = fetch_20newsgroups(subset='train', categories=categories)
data = newsgroups['data']

NameError: name 'fetch_20newsgroups' is not defined

In [6]:
data

['From: bil@okcforum.osrhe.edu (Bill Conner)\nSubject: Re: Americans and Evolution\nNntp-Posting-Host: okcforum.osrhe.edu\nOrganization: Okcforum Unix Users Group\nX-Newsreader: TIN [version 1.1 PL9]\nLines: 30\n\nRobert Singleton (bobs@thnext.mit.edu) wrote:\n\n: > Sure it isn\'t mutually exclusive, but it lends weight to (i.e. increases\n: > notional running estimates of the posterior probability of) the \n: > atheist\'s pitch in the partition, and thus necessarily reduces the same \n: > quantity in the theist\'s pitch. This is because the `divine component\' \n: > falls prey to Ockham\'s Razor, the phenomenon being satisfactorily \n:                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^\n: > explained without it, and there being no independent evidence of any \n:   ^^^^^^^^^^^^^^^^^^^^\n: > such component. More detail in the next post.\n: > \n\nOccam\'s Razor is not a law of nature, it is way of analyzing an\nargument, even so, it interesting how often it\'s cited here

In [7]:
# CountVectorizer
vocabVect = CountVectorizer()
vocabVect.fit(data)
corpusVocab = vocabVect.vocabulary_
print('Количество сформированных признаков - {}'.format(len(corpusVocab)))

Количество сформированных признаков - 31660


In [8]:
for i in list(corpusVocab)[1:10]:
    print('{}={}'.format(i, corpusVocab[i]))

bil=6823
okcforum=21419
osrhe=21683
edu=11993
bill=6827
conner=9374
subject=27588
re=24066
americans=5315


In [9]:
test_features = vocabVect.transform(data)
test_features

<2259x31660 sparse matrix of type '<class 'numpy.int64'>'
	with 310255 stored elements in Compressed Sparse Row format>

In [10]:
# Размер нулевой строки
len(test_features.todense()[0].getA1())

31660

In [11]:
# Непустые значения нулевой строки
[i for i in test_features.todense()[0].getA1() if i>0]

[1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 5,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 3,
 1,
 1,
 1,
 1,
 7,
 1,
 8,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 8,
 1,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 11,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2]

In [12]:
vocabVect.get_feature_names()[100:120]

['0216',
 '021635',
 '022',
 '022233',
 '022621tan102',
 '022922',
 '023044',
 '023428',
 '0235',
 '0239',
 '024',
 '024036',
 '024150',
 '02451203',
 '024626',
 '025924',
 '02678944',
 '02790',
 '0280',
 '0284']

In [13]:
def VectorizeAndClassify(vectorizers_list, classifiers_list):
    for v in vectorizers_list:
        for c in classifiers_list:
            pipeline1 = Pipeline([("vectorizer", v), ("classifier", c)])
            score = cross_val_score(pipeline1, newsgroups['data'], newsgroups['target'], scoring='accuracy', cv=3).mean()
            print('Векторизация - {}'.format(v))
            print('Модель для классификации - {}'.format(c))
            print('Accuracy = {}'.format(score))
            print('===========================')

In [14]:
vectorizers_list = [CountVectorizer(vocabulary = corpusVocab), TfidfVectorizer(vocabulary = corpusVocab)]
classifiers_list = [LogisticRegression(C=3.0), LinearSVC(), KNeighborsClassifier()]
VectorizeAndClassify(vectorizers_list, classifiers_list)

c:\users\arthur\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\users\arthur\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sc

Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '00000': 3,
                            '0000000004': 4, '0000000005': 5, '0000000667': 6,
                            '0000001200': 7, '000042': 8, '000094': 9,
                            '0001': 10, '0001x7c': 11, '0002': 12, '0003': 13,
                            '000406': 14, '0005111312': 15,
                            '0005111312na1em': 16, '000531': 17, '00072': 18,
                            '000851': 19, '000rpm': 20, '001': 21,
                            '00100111b': 22, '0011': 23, '001125': 24,
                            '0013': 25, '001813': 26, '002222': 27,
                            '002937': 28, '003029': 29, ...})
Модель для классификации - LogisticRegression(C=3.0)
Accuracy = 0.923860115095175


c:\users\arthur\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\arthur\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '00000': 3,
                            '0000000004': 4, '0000000005': 5, '0000000667': 6,
                            '0000001200': 7, '000042': 8, '000094': 9,
                            '0001': 10, '0001x7c': 11, '0002': 12, '0003': 13,
                            '000406': 14, '0005111312': 15,
                            '0005111312na1em': 16, '000531': 17, '00072': 18,
                            '000851': 19, '000rpm': 20, '001': 21,
                            '00100111b': 22, '0011': 23, '001125': 24,
                            '0013': 25, '001813': 26, '002222': 27,
                            '002937': 28, '003029': 29, ...})
Модель для классификации - LinearSVC()
Accuracy = 0.9251881363435147
Векторизация - CountVectorizer(vocabulary={'00': 0, '000': 1, '0000': 2, '00000': 3,
                            '0000000004': 4, '0000000005': 5, '0000000667': 6,
                            '0000001200': 7, '0

In [15]:
# word2vec 
# Подготовим корпус
corpus = []
stop_words = stopwords.words('english')
tok = WordPunctTokenizer()
for line in newsgroups['data']:
    line1 = line.strip().lower()
    line1 = re.sub("[^a-zA-Z]"," ", line1)
    text_tok = tok.tokenize(line1)
    text_tok1 = [w for w in text_tok if not w in stop_words]
    corpus.append(text_tok1)

In [16]:
corpus[:5]

[['bil',
  'okcforum',
  'osrhe',
  'edu',
  'bill',
  'conner',
  'subject',
  'americans',
  'evolution',
  'nntp',
  'posting',
  'host',
  'okcforum',
  'osrhe',
  'edu',
  'organization',
  'okcforum',
  'unix',
  'users',
  'group',
  'x',
  'newsreader',
  'tin',
  'version',
  'pl',
  'lines',
  'robert',
  'singleton',
  'bobs',
  'thnext',
  'mit',
  'edu',
  'wrote',
  'sure',
  'mutually',
  'exclusive',
  'lends',
  'weight',
  'e',
  'increases',
  'notional',
  'running',
  'estimates',
  'posterior',
  'probability',
  'atheist',
  'pitch',
  'partition',
  'thus',
  'necessarily',
  'reduces',
  'quantity',
  'theist',
  'pitch',
  'divine',
  'component',
  'falls',
  'prey',
  'ockham',
  'razor',
  'phenomenon',
  'satisfactorily',
  'explained',
  'without',
  'independent',
  'evidence',
  'component',
  'detail',
  'next',
  'post',
  'occam',
  'razor',
  'law',
  'nature',
  'way',
  'analyzing',
  'argument',
  'even',
  'interesting',
  'often',
  'cited',
  

In [17]:
%time model_data = word2vec.Word2Vec(corpus, workers=4, min_count=10, window=10, sample=1e-3)

Wall time: 2.96 s


In [18]:
def sentiment(v, c):
    model = Pipeline(
        [("vectorizer", v), 
         ("classifier", c)])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print_accuracy_score_for_classes(y_test, y_pred)

In [19]:
class EmbeddingVectorizer(object):
    '''
    Для текста усредним вектора входящих в него слов
    '''
    def __init__(self, model):
        self.model = model
        self.size = model.vector_size

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([np.mean(
            [self.model[w] for w in words if w in self.model] 
            or [np.zeros(self.size)], axis=0)
            for words in X])

In [20]:
# Обучающая и тестовая выборки
boundary = 700
X_train = corpus[:boundary] 
X_test = corpus[boundary:]
y_train = newsgroups['target'][:boundary]
y_test = newsgroups['target'][boundary:]

In [21]:
sentiment(EmbeddingVectorizer(model_data.wv), LogisticRegression(C=5.0))

c:\users\arthur\appdata\local\programs\python\python36\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Метка 	 Accuracy
0 	 0.9501557632398754
1 	 0.723404255319149
2 	 0.8329177057356608
3 	 0.7729468599033816
